# Multi-Purpose Agentic Assistant with RAG, Evaluation, Human Input, Web Search, and Python Tool

## Motivation

How do we build a helpful assistant that has the ability to search over many different internal knowledge bases and web? Can the assistant do math? Instead of having to build separate assistants for different use cases such as IT assistant, HR assistant, general LLM, or Python interpreter, you can equip agents with multiple tools so that you can build a single system that can handle different use cases. Based on user query, the LLM can decide what tool to use.

Useful and capable assistant should have access to multiple knowledge bases such as Slack, Google Drive, Notion, Jira, and more. In this situation, you can either build a single database with multiple data sources or handle each one separately depending on the requirement and complexity of combining multiple sources. In this notebook, we showcase how we give the model different tool per different data source so that the model can query relevant databases.

Then it evaluates retrieved documents using a modified version of [Correctness from LlamaIndex](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/evaluation/correctness.py) evaluator template. If the score is low, it asks the user for permission to do a web search to find the answer.

Depending on the question, the agent also has the ability to use Python to handle user's question.

## Objective

This notebook will showcase how an agent can equip LLM with multiple tools like RAG and web search to become a useful assistant.

## Questions

- "How much do we get for a referral?" -> HR question
- "How do I set up api key for cohere?" -> IT Question
- "Is cohere available on aws" -> web search question
- "How much do I get a year on learning and development, and can you calculate how much I can spend per week?" --> HR question + Calculation

## Data

- Mock internal database of HR documents
- Mock internal database of IT documents (such as API docs)

## Tools

- Python Interpreter
- HR data retriever
- IT data retriever
- Web search tool with human permission


In [111]:
import os
from pprint import pprint

import cohere
import pandas as pd
from langchain.agents import Tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.utilities import PythonREPL
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# uncomment if you need to install the following packages
# !pip install --quiet langchain langchain_experimental cohere --upgrade

In [113]:
# versions
print('cohere version:', cohere.__version__)
print('langchain version:', langchain.__version__)
print('langchain_core version:', langchain_core.__version__)
print('langchain_experimental version:', langchain_experimental.__version__)

cohere version: 5.5.1
langchain version: 0.2.0
langchain_core version: 0.2.0
langchain_experimental version: 0.0.59


## Setup


In [7]:
COHERE_API_KEY = os.environ.get("CO_API_KEY")
COHERE_MODEL = 'command-r-plus'
co = cohere.Client(api_key=COHERE_API_KEY)

## Data

Here we define HR and IT related documents, which will be used as a mock database.


In [100]:
# generated by LLM
hr_documents = [
    {
        "title": "Remote Work Policy",
        "content": "We embrace a remote-friendly work environment, allowing employees to work remotely on a full-time or hybrid basis. However, employees should expect to come to the office once a quarter. Expectations for remote work include maintaining regular working hours, responding promptly to communications, and attending virtual meetings. We provide remote workers with the necessary equipment and reimburse expenses for setting up a home office. Virtual collaboration tools, such as video conferencing software and project management platforms, are utilized to ensure effective remote work experiences. In-person meetings and team-building activities are also organized periodically to foster connections.",
    },
    {
        "title": "Global Mobility Program",
        "content": "Employees have the opportunity to work in different countries through our Global Mobility Program. This policy outlines the process for international assignments, including permanent transfers, short-term projects, and rotational programs. We provide support with visa sponsorship, relocation assistance, and cultural integration. Compensation packages are adjusted to align with the host country's market rates, and employees are offered pre-assignment training and ongoing support during their time abroad. Reassignment back to their home country or another location is also facilitated as part of this program. ",
    },
    {
        "title": "Paid Time Off and Sick Days",
        "content": "We offer a competitive paid time off package, including vacation days, personal days, and sick leave. Employees are entitled to a set number of paid vacation days each year, which increases with tenure. Unlimited sick days are provided to ensure employees can take time off for their health and well-being. Additionally, we offer paid parental leave, bereavement leave, and volunteer time off. Our policy also outlines procedures for requesting and tracking time off, as well as guidelines for managing unused days, carry-over limits, and payout options.",
    },
    {
        "title": "Learning and Development Reimbursement",
        "content": "Employees are eligible up to $9000 per year for learning and development. Investing in our employees' growth, we offer a learning and development reimbursement policy. Employees can seek reimbursement for work-related courses, certifications, conferences, and training programs. This includes tuition fees, course materials, and travel expenses associated with attending educational events.",
    },
    {
        "title": "Employee Referral Program",
        "content": "We value employee referrals and have implemented a referral bonus program. Employees who refer successful hires are eligible for a monetary bonus upon the referred candidate's start date. Our policy outlines the bonus amounts, eligibility criteria, and the referral process. We also offer incentives for referring diverse talent and provide employees with resources and guidance on effective referral strategies, including access to networking events and referral training. We offer $5000 for every successful referral.",
    },
]

# from https://github.com/cohere-ai/cohere-python/blob/main/README.md
it_documents = [
    {
        "title": "Cohere SDK Streaming",
        "content": """The SDK supports streaming endpoints. To take advantage of this feature for chat,
        use `chat_stream`.

        ```Python
        import cohere

        co = cohere.Client(
            api_key="YOUR_API_KEY",
        )

        stream = co.chat_stream(
            message="Tell me a short story"
        )

        for event in stream:
            if event.event_type == "text-generation":
                print(event.text, end='')
        ```""",
    },
    {
        "title": "Cohere SDK environment variable",
        "content": """> [!TIP]
        > You can set a system environment variable `CO_API_KEY` to avoid writing your api key within your code, e.g. add `export CO_API_KEY=theapikeyforyouraccount`
        > in your ~/.zshrc or ~/.bashrc, open a new terminal, then code calling `cohere.Client()` will read this key.
        """,
    },
]


The following function helps us to convert IT and HR documents as a pandas dataframe with embeddings.


In [101]:
def dbify(db):
    """
    Convert a list of dictionaries to a pandas DataFrame and add embeddings to be used as a mock database
    """
    db = pd.DataFrame(db)
    # comebine title and body
    db["combined"] = "Title: " + db["title"] + "\n" + "Body: " + db["content"]
    # generate embedding
    embeddings = co.embed(
        texts=db.combined.tolist(),
        model="embed-english-v3.0",
        input_type="search_document",
    )
    db["embeddings"] = embeddings.embeddings
    return db


db_it = dbify(it_documents)
db_hr = dbify(hr_documents)


## Tools

Define tools that are used by the agent.


### Retriever tools

Define tools related to retrieval:

- evaluator: evaluates the quality of the retrieved document.
- web_search: performs web search given query.
- retrieve_documents: retrieves top matching documents from a database.


In [105]:
def evaluator(query, retrieved_documents):
    criteria = """
    You are an expert evaluation system for a question answering chatbot.

    You are given the following information:
    - a user query, and
    - a generated answer

    Your job is to judge the relevance and correctness of the generated answer.
    Output a single score that represents a holistic evaluation.
    You must return your response in a line with only the score.
    Do not return answers in any other format.

    Follow these guidelines for scoring:
    - Your score has to be between 1 and 5, where 1 is the worst and 5 is the best.
    - If the generated answer is not relevant to the user query, \
    you should give a score of 1.
    - If the generated answer is relevant but does not fully answer the question, \
    you should give a score between 2 and 3.
    - If the generated answer is relevant and fully correct, \
    you should give a score between 4 and 5.
    """

    prompt = f"""
    ## User Query
    {query}

    ## Retrieved Documents
    {retrieved_documents}

    ## Criteria
    {criteria}

    ## Output format
    Ouput a single score that represents a holistic evaluation.
    """
    return co.chat(message=prompt, model=COHERE_MODEL, preamble=None).text


def web_search(query):
    """
    Function to search the web for a given query.
    """
    question = "I could not find relevant information in the database. Do you want me to search the web? \nPlease enter 'y' or 'n':"

    while True:
        response = input(question)
        if response == "y":
            print("You entered 'y'.")
            response = co.chat(
                message=query,
                connectors=[{"id": "web-search"}],
            )
            return {"web_result": response.text}
        elif response == "n":
            print("You entered 'n'.")
            return {
                "result": "User declined to search the web. Complete the conversation."
            }
        else:
            print("Invalid input. Please enter 'y' or 'n'.")


def retrieve_documents(query: str, db, n):
    """
    Function to retrieve most relevant documents a given query.
    """

    query_emb = co.embed(
        texts=[query], model="embed-english-v3.0", input_type="search_query"
    )

    similarity_scores = cosine_similarity(
        [query_emb.embeddings[0]], db.embeddings.tolist()
    )
    similarity_scores = similarity_scores[0]

    top_indices = similarity_scores.argsort()[::-1][:n]
    top_matches = db.iloc[top_indices]

    evaluator_score = float(evaluator(query, top_matches.combined.tolist()))

    if evaluator_score >= 4:
        status_message = "Success: Retrieved documents are relevant and correct. Please answer user's question."
    else:
        status_message = (
            "Warning: Retrieved documents are not relevant, please search the web."
        )

    return {
        "top_matched_document": top_matches.combined,
        "evaluator_score": evaluator_score,
        "status_message": status_message,
    }


def retrieve_it_documents(query: str, db=db_it, n=2) -> dict:
    """
    Function to retrieve most relevant documents a given query.
    It also returns other references mentioned in the top matched documents.
    """
    return retrieve_documents(query, db, n)


def retrieve_hr_documents(query: str, db=db_hr, n=2) -> dict:
    """
    Function to retrieve most relevant documents a given query.
    It also returns other references mentioned in the top matched documents.
    """
    return retrieve_documents(query, db, n)


### Python tool


In [ ]:
python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
python_tool.name = "python_interpreter"


class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")


python_tool.args_schema = ToolInput


def run_python_code(code: str) -> dict:
    """
    Function to run given python code
    """
    input_code = ToolInput(code=code)
    return {"python_answer": python_tool.func(input_code.code)}


### Functions map

Define mapping of functions and function definitions for the agent to refer.


In [ ]:

functions_map = {
    "retrieve_it_documents": retrieve_it_documents,
    "retrieve_hr_documents": retrieve_hr_documents,
    "web_search": web_search,
        "run_python_code": run_python_code,

}

tools = [
    {
        "name": "retrieve_it_documents",
        "description": "given a query, retrieve documents from a database to answer user's question related to IT",
        "parameter_definitions": {
            "query": {
                "description": "user's question or query",
                "type": "str",
                "required": True,
            }
        },
    },
    {
        "name": "retrieve_hr_documents",
        "description": "given a query, retrieve documents from a database to answer user's question related to HR.",
        "parameter_definitions": {
            "query": {
                "description": "user's question or query",
                "type": "str",
                "required": True,
            }
        },
    },
    {
        "name": "web_search",
        "description": "Search web to answer user's queston",
        "parameter_definitions": {
            "query": {
                "description": "user's question or query",
                "type": "str",
                "required": True,
            }
        },
    },
 {
        "name": "run_python_code",
        "description": "given a python code, runs it",
        "parameter_definitions": {
            "code": {
                "description": "executable python code",
                "type": "str",
                "required": True
            }
        }
    },

]


## Cohere Agent

Wrapper of Cohere API to handle multi step tool use.


In [112]:
def cohere_agent(
    message: str,
    preamble: str,
    tools: list[dict],
    force_single_step=False,
    verbose: bool = False,
    temperature: float = 0.3,
) -> str:
    """
    Function to handle multi-step tool use api.

    Args:
        message (str): The message to send to the Cohere AI model.
        preamble (str): The preamble or context for the conversation.
        tools (list of dict): List of tools to use in the conversation.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        str: The final response from the call.
    """

    counter = 1

    response = co.chat(
        model=COHERE_MODEL,
        message=message,
        preamble=preamble,
        tools=tools,
        force_single_step=force_single_step,
        temperature=temperature,
    )

    if verbose:
        print(f"\nrunning 0th step.")
        print(response.text)

    while response.tool_calls:
        tool_results = []

        if verbose:
            print(f"\nrunning {counter}th step.")

        for tool_call in response.tool_calls:
            if tool_call.parameters:
                output = functions_map[tool_call.name](**tool_call.parameters)
            else:
                output = functions_map[tool_call.name]()

            outputs = [output]
            tool_results.append({"call": tool_call, "outputs": outputs})

            if verbose:
                print(
                    f"= running tool {tool_call.name}, with parameters: \n{tool_call.parameters}"
                )
                print(f"== tool results:")
                pprint(output)

        response = co.chat(
            model=COHERE_MODEL,
            message="",
            chat_history=response.chat_history,
            preamble=preamble,
            tools=tools,
            force_single_step=force_single_step,
            tool_results=tool_results,
            temperature=temperature,
        )

        if verbose:
            print(response.text)
            counter += 1

    return response.text


## Preamble

Preamble is a system level instruction that the agent follow.


In [78]:
preamble = """
You are helpful assitant for employees that has access to multiple databases such as HR and IT.
Search relevant databases first. If you cannot find relevant information, search the web.
You may need to use python to run some code or make calculations.

Walk me through each step on what you are considering and going to do.
"""

## Questions

List of questions to ask the agent.


In [ ]:

questions = [
    "how much do we get for a referral?",
    "how do I set up api key for cohere?",
    "Is cohere available on aws",
    "how much do I get a year on learning and development. and can you calculate how much I can spend per week?"
]

### Question 1 - "how much do we get for a referral?"


In [107]:
output = cohere_agent(questions[0], preamble, tools, verbose=True)


running 0th step.
I will search the HR database for information about referral bonuses.

running 1th step.
= running tool retrieve_hr_documents, with parameters: 
{'query': 'referral bonus'}
== tool results:
{'evaluator_score': 5.0,
 'status_message': 'Success: Retrieved documents are relevant and correct. '
                   "Please answer user's question.",
 'top_matched_document': 4    Title: Employee Referral Program\nBody: We val...
3    Title: Learning and Development Reimbursement\...
Name: combined, dtype: object}
Employees who refer successful hires are eligible for a bonus of $5000 upon the referred candidate's start date. We also offer incentives for referring diverse talent and provide employees with resources and guidance on effective referral strategies, including access to networking events and referral training.


### Question 2 - "how do I set up api key for cohere?"


In [108]:
output = cohere_agent(questions[1], preamble, tools, verbose=True)


running 0th step.
I will search the IT database for information on how to set up an API key for Cohere.

running 1th step.
= running tool retrieve_it_documents, with parameters: 
{'query': 'how to set up api key for cohere'}
== tool results:
{'evaluator_score': 5.0,
 'status_message': 'Success: Retrieved documents are relevant and correct. '
                   "Please answer user's question.",
 'top_matched_document': 1    Title: Cohere SDK environment variable\nBody: ...
0    Title: Cohere SDK Streaming\nBody: The SDK sup...
Name: combined, dtype: object}
To set up an API key for Cohere, you can use the following code:

```Python
import cohere

co = cohere.Client(
 api_key="YOUR_API_KEY",
)
```

You can also set a system environment variable `CO_API_KEY` to avoid writing your API key within your code. For example, add `export CO_API_KEY=theapikeyforyouraccount` in your `~/.zshrc` or `~/.bashrc`, open a new terminal, then code calling `cohere.Client()` will read this key.


### Quiestion 3 - "Is cohere available on aws"


In [79]:
# gives permission to search the web
output = cohere_agent(questions[2], preamble, tools, verbose=True)


running 0th step.
I will search the databases for information on whether Cohere is available on AWS.

running 1th step.
= running tool retrieve_it_documents, with parameters: 
{'query': 'Is cohere available on aws?'}
== tool results:
{'evaluator_score': 1.0,
 'status_message': 'Warning: Retrieved documents are not relevant, please ask '
                   'for human feedback.',
 'top_matched_document': 0    Title: Cohere SDK Streaming\nBody: The SDK sup...
1    Title: Cohere SDK environment variable\nBody: ...
Name: combined, dtype: object}
I could not find any relevant information in the databases. I will now search the web for the answer.

running 2th step.
I could not find relevant information in the database. Do you want me to search the web? 
Please enter 'y' or 'n':
You entered 'y'.
= running tool web_search, with parameters: 
{'query': 'Is cohere available on aws?'}
== tool results:
{'web result': 'Yes, Cohere is available on AWS. Developers can access a range '
               

In [80]:
# does not give permission to search the web
output = cohere_agent(questions[2], preamble, tools, verbose=True)


running 0th step.
I will search the databases for information on whether Cohere is available on AWS.

running 1th step.
= running tool retrieve_it_documents, with parameters: 
{'query': 'Is cohere available on aws?'}
== tool results:
{'evaluator_score': 1.0,
 'status_message': 'Warning: Retrieved documents are not relevant, please ask '
                   'for human feedback.',
 'top_matched_document': 0    Title: Cohere SDK Streaming\nBody: The SDK sup...
1    Title: Cohere SDK environment variable\nBody: ...
Name: combined, dtype: object}
I could not find any relevant information in the databases. I will now search the web for the answer.

running 2th step.
I could not find relevant information in the database. Do you want me to search the web? 
Please enter 'y' or 'n':
You entered 'n'.
= running tool web_search, with parameters: 
{'query': 'Is cohere available on aws?'}
== tool results:
{'result': 'User declined to search the web. Complete the conversation.'}
I'm sorry, I can't ans

### Question 4 - "how much do I get a year on learning and development. and can you calculate how much I can spend per week?"


In [106]:
output = cohere_agent(questions[3], preamble, tools, verbose=True)


running 0th step.
I will first search the HR database for information on how much the user gets a year on learning and development. Then, I will calculate how much they can spend per week.

running 1th step.
= running tool retrieve_hr_documents, with parameters: 
{'query': 'how much do I get a year on learning and development'}
== tool results:
{'evaluator_score': 5.0,
 'status_message': 'Success: Retrieved documents are relevant and correct. '
                   "Please answer user's question.",
 'top_matched_document': 3    Title: Learning and Development Reimbursement\...
4    Title: Employee Referral Program\nBody: We val...
Name: combined, dtype: object}
I have found that employees are eligible for up to $9000 per year for learning and development. Now, I will calculate how much they can spend per week.

running 2th step.
= running tool run_python_code, with parameters: 
{'code': 'yearly_budget = 9000\n\nweekly_budget = yearly_budget / 52\n\nprint(f"You can spend up to ${weekly_b

## Conclusion


You have now seen how the agent can be a useful assistant by combining multiple tools to answer questions. The agent can search databases, search the web, and run python code to answer questions.
